# COLIN API Utils for getting filings

In [ ]:
import os
from http import HTTPStatus

import requests
from colin_api.models.filing import Filing as ColinFiling
from dotenv import find_dotenv, load_dotenv
from flask import Flask
from legal_api import db
from legal_api.config import get_named_config
from legal_api.models import (
    Business,
    Filing,
)
from legal_api.services import queue
from legal_api.models.colin_event_id import ColinEventId

In [ ]:
COLIN_API = os.getenv('COLIN_API')
TIMEOUT = os.getenv('TIMEOUT', 30)


In [ ]:
def echo_colin_api():
    print (f'{COLIN_API}')
    
def create_filing(filing_type, colin_filing, colin_event_id, corp_num):
    """Create legal api filing using colin filing as base"""
    effective_date = colin_filing['filing']['business']['foundingDate']
    colin_filing['filing']['business']['identifier'] = corp_num
    filing = Filing(effective_date=effective_date, filing_json=colin_filing)
    filing._filing_type = filing_type
    filing.filing_date = effective_date
    colin_event = ColinEventId()
    colin_event.colin_event_id = colin_event_id
    filing.colin_event_ids.append(colin_event)

    # Override the state setting mechanism
    filing.skip_status_listener = True
    filing._status = 'PENDING'
    filing.source = Filing.Source.COLIN.value
    return filing

def get_data_load_required_filing_event(legal_type, corp_num):
    """Determine whether corp has required filings types (incorp app or conversion ledger)"""
    events = get_filing_events_for_corp(legal_type, corp_num)
    match = next((x for x in events if x.get('filing_typ_cd')
                  in ['OTINC', 'BEINC', 'ICORP', 'ICORU', 'ICORC', 'CONVL']), None)
    return match

def get_filing_events_for_corp(legal_type, corp_num):
    """Retrieve filing events for a given corp"""
    colin_corp_num = corp_num
    if(legal_type == Business.LegalTypes.COMP.value):
        colin_corp_num = corp_num[-7:]

    r = requests.get(f'{COLIN_API}/api/v1/businesses/event/corp_num/{colin_corp_num}', timeout=TIMEOUT)
    if r.status_code != HTTPStatus.OK or not r.json():
        return None

    events = dict(r.json()).get('events', [])
    return events

def get_filing(colin_filing_type, legal_type, event_info: dict = None):  # pylint: disable=redefined-outer-name
    """Get filing for a given event from colin"""
    identifier = event_info['corp_num']
    event_id = event_info['event_id']
    print(f'{COLIN_API}/api/v1/businesses/{legal_type}/{identifier}/filings/{colin_filing_type}?eventId={event_id}')
    response = requests.get(
        f'{COLIN_API}/api/v1/businesses/{legal_type}/{identifier}/filings/{colin_filing_type}?eventId={event_id}'
    )
    print(response.json())
    filing = dict(response.json())
    return filing

def get_filing_type(filing_typ_cd):
    """Get generic filing type """
    filing_types = ColinFiling.FILING_TYPES.keys()
    match = next((x for x in filing_types
                  if filing_typ_cd in ColinFiling.FILING_TYPES.get(x).get('type_code_list')),
                 None)
    return match